# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119175e+02     1.651349e+00
 * time: 0.0812830924987793
     1     1.094624e+01     9.453336e-01
 * time: 1.517003059387207
     2    -1.186942e+01     1.090915e+00
 * time: 1.6210041046142578
     3    -3.394425e+01     8.061581e-01
 * time: 1.7613279819488525
     4    -4.757878e+01     5.210986e-01
 * time: 1.9192180633544922
     5    -5.680177e+01     1.956414e-01
 * time: 2.0414891242980957
     6    -5.969963e+01     1.375643e-01
 * time: 2.1398980617523193
     7    -6.084425e+01     4.949866e-02
 * time: 2.2605371475219727
     8    -6.130275e+01     5.703886e-02
 * time: 2.356412172317505
     9    -6.157906e+01     4.020241e-02
 * time: 2.4536800384521484
    10    -6.180778e+01     3.027354e-02
 * time: 2.548866033554077
    11    -6.198677e+01     2.250260e-02
 * time: 2.656860113143921
    12    -6.206203e+01     1.654034e-02
 * time: 2.7503480911254883
    13    -6.212883e+01     1.518655e-02
 * time: 2.84437108039855

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671067
    AtomicLocal         -18.8557689
    AtomicNonlocal      14.8522658
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485371 
    Xc                  -19.3336820

    total               -62.261666460135
